In [1]:
from DenoisingDiffusionPM import EMA, LinearNoiseScheduler, NoisePredictor
from DenoisingDiffusionPM import Dataset, DDPM, Architecture
import torch
import numpy as np


In [2]:
betas = torch.linspace(1e-4, 2e-2, 1000)
num_dims_to_add = 2
betas2 = torch.linspace(1e-4, 2e-2, 1000).view(*([-1] + [1]*num_dims_to_add))

# print memory usage of betas and betas2 in MB
print(betas.element_size() * betas.nelement() / 1024**2)
print(betas2.element_size() * betas2.nelement() / 1024**2)
# print the shape of betas and betas2
print(betas.shape)
print(betas2.shape)

0.003814697265625
0.003814697265625
torch.Size([1000])
torch.Size([1000, 1, 1])


In [3]:
dataset = Dataset()
dataloader = dataset.generate_data()
dataset_shape = dataset.get_dataset_shape()

In [4]:
dataset_shape 

(1200, 2)

In [5]:
scheduler = LinearNoiseScheduler(num_timesteps=100, dataset_shape=dataset_shape)

In [6]:
scheduler.betas.shape

torch.Size([100, 1])

In [7]:
# extract a sample from the dataloader
x, y = next(iter(dataloader))

In [8]:
x.shape

torch.Size([14, 2])

In [9]:
t = torch.randint(0, scheduler.num_timesteps, (x.shape[0],))
print(t.shape)

noise = torch.randn_like(x)
print(noise.shape)

torch.Size([14])
torch.Size([14, 2])


In [10]:
x_t = scheduler.add_noise(x, noise, t)
x_t

tensor([[ 2.1914, -0.7415],
        [-3.0195,  4.1023],
        [ 3.4742, -1.6270],
        [ 6.1760, -3.9222],
        [ 8.9461,  7.4753],
        [-4.4781,  6.1663],
        [-3.2277, -3.4854],
        [-2.5680,  3.8849],
        [ 6.3882,  5.8954],
        [ 7.6141,  9.5528],
        [ 2.1188, -3.1247],
        [-3.2778, -3.9302],
        [ 5.0305, -2.0355],
        [-3.9435,  5.7470]])

In [11]:
architecture = Architecture(256, dataset_shape)

In [12]:
# architecture.emb_layer(t)

In [13]:
model = NoisePredictor(time_dim = 256, dataset_shape = dataset_shape , num_classes=2)
# model(x_t, t, y)

In [14]:
# t = t.unsqueeze(-1)
# model.positional_encoding(t, 256)

In [15]:
# from torch import nn
# label_embedder = nn.Embedding(2, 256)
# print(y.shape) # (14, 1)
# if y.dtype!= torch.long:
#     y = y.long()

# y_embedded = label_embedder(y)
# print(y_embedded.shape)


In [16]:
t.shape

torch.Size([14])

In [17]:
# t = t.unsqueeze(-1)
# t = model.positional_encoding(t, 256)
# t.shape

In [18]:
# (t + y_embedded).shape

In [22]:
# # print dimenstions of x_t, t, y
print(x_t.shape)
print(t.shape)
print(y.squeeze().shape)

torch.Size([14, 2])
torch.Size([14])
torch.Size([14])


In [23]:
model(x_t, t, y)

torch.Size([14, 256])
torch.Size([14, 256])


tensor([[ 0.1666,  0.9483],
        [ 0.1144,  0.5371],
        [ 0.3037,  1.2256],
        [ 0.6358,  1.8971],
        [ 0.4528,  1.5270],
        [ 0.4130,  0.8305],
        [-0.1290,  0.2980],
        [ 0.0845,  0.5078],
        [ 0.2733,  1.1641],
        [ 0.3240,  1.2667],
        [ 0.2448,  1.1064],
        [-0.1290,  0.2980],
        [ 0.4884,  1.5990],
        [ 0.3577,  0.7761]], grad_fn=<AddmmBackward0>)

In [29]:
t_embedded = model.positional_encoding(t.unsqueeze(-1), 256)
t_embedded.shape

torch.Size([14, 256])

In [32]:
emb = architecture.emb_layer(t_embedded)
emb.shape

torch.Size([14, 2])

In [33]:
x.shape

torch.Size([14, 2])